# Overview

Graph Neural Networks have emberged as a powerful class of neural networks, desgined to capture the complexity and relational information inherent in graph-structured data. Much like traditional neural networks, onions and ogres, GNNs are composed of layers.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCNLayer(nn.Module):
    r"""
    Args:
        input_dim(int): Dimension of the input
        output_dim(iunt): Dimension of the output (a softmax distribution)
        A (torch.Tensor): 2D adjacency matrix
    """
    
    def __init__(self, input_dim, output_dim, A):
        super(GCNLayer, self).__init__()
        self.input_dim=input_dim
        self.output_dim=output_dim
        self.A=A
        
        #A_hat=A+I
        self.A_hat=self.A+torch.eye(self.A.size(0))
        
        # create diagonal degree matrix D
        self.ones=torch.ones(input_dim, input_dim)
        self.D=torch.matmul(self.A.float(), self.ones.float())
        
        # Extract the diagonal elements
        self.D=torch.diag(self.D)
        
        # Create a new tensor with the diagonal elements and zeros elsewhere
        self.D=torch.diag_embed(self.D)
        
        # create D^{-1/2}
        self.D_neg_sqrt=torch.diag_embed(torch.diag(torch.pow(self.D, -0.5)))
        
        # Initialise the weight matrix as a parameter
        self.W=nn.Parameter(torch.rand(input_dim, output_dim))
        
    def forward(self,X):
        # D^-1/2 * (A_hat *D^-1/2)
        support_1=torch.matmul(self.D_neg_sqrt, torch.matmul(self.A_hat, self.D_neg_sqrt))
        
        # (D^-1/2 * A_hat * D^-1/2)*(X*W)
        support_2=torch.matmul(support_1, torch.matmul(X, self.W))
        
        # ReLU(D^-1/2 * A_hat * D^-1/2 *X*W)
        H=F.relu(support_2)
        
        return H
    
    
# example usage
input_dim=3
output_dim=2
    
A=torch.tensor([[1.,0.,0.],[0.,1.,1.],[0.,1.,1.]])
    
gcn_layer=GCNLayer(input_dim, output_dim, A)
    
X=torch.tensor([[1.,2.,3.],[4.,5.,6.],[7.,8.,9.]])
    
output=gcn_layer(X)

output

tensor([[ 6.2169,  6.4498],
        [12.5922, 14.3301],
        [14.5746, 16.7033]], grad_fn=<ReluBackward0>)

# Acknowledge

* https://medium.com/@jrosseruk/demystifying-gcns-a-step-by-step-guide-to-building-a-graph-convolutional-network-layer-in-pytorch-09bf2e788a51